In [ ]:
%run ../../Setup.ipynb
import ast
import json
import pandas as pd
import numpy as np
from bdcc.database.connection import database_connector as connector
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [ ]:
def write_match(match):
    """
    Function to create entries in the database.
    params: *match: dict of match data
    return: HTTP-Status Code
    """
    http_status = 200   
    db_con = connector.DatabaseConnector() # connect to database
    db_con.connect()

    match_found = False
    for entry in db_con.get(id=match['match_id']):
        if entry:
            match_found = True

    if not match_found:
        http_status = 201
        db_con.create(match) # create new entry in database
    else:
        http_status = 409

    db_con.disconnect()

    return http_status

In [ ]:
def delete_match(match_id):
    """
    Function to delete entries in the database.
    params: *match_id: id of the match to be deleted
    """
    http_status = 200
    db_con = connector.DatabaseConnector()
    db_con.connect()

    # Überprüfung, ob zu löschendes Match in Datenbank existiert
    matches = list(db_con.get())
    if any(match['match_id'] == match_id for match in matches):
        http_status = 205
        db_con.remove(match_id)
    else:
        http_status = 204
    
    db_con.disconnect()
    return http_status

In [ ]:
def build_dataframe_no_kda(matches):
    """
    Function to build dataframe for model no_kda.
    params: *matches: match data
    """
    ttl_pings_list = []
    
    for j in range(len(matches)):
        total_pings_win = 0
        total_pings_loss = 0
        for i in range(len(matches[j]['players'])):
            if matches[j]['players'][i]['win'] == True:
                total_pings_win += matches[j]['players'][i]['pings']
            else:
                total_pings_loss += matches[j]['players'][i]['pings']
        ttl_pings_list.append([total_pings_win, True])
        ttl_pings_list.append([total_pings_loss, False])
    dataframe = pd.DataFrame(ttl_pings_list, columns =['ttlPings', 'win'])
    dataframe = dataframe.dropna() # just to be sure
    
    return dataframe

In [ ]:
def build_dataframe_kda(matches):
    """
    Function to build dataframe for model kda.
    params: *matches: match data
    """
    filter_keys = ['assists', 'deaths', 'kills', 'pings', 'win']
    player_kda_list = []
    
    for j in range(len(matches)):
        for i in range(len(matches[j]['players'])):
            player_kda_list.append({key:value for key, value in matches[j]['players'][i].items() if key in filter_keys})

    dataframe_kda = pd.DataFrame(player_kda_list)
    dataframe_kda = dataframe_kda.dropna() # just to be sure
    
    return dataframe_kda

In [ ]:
def train_model(model_name):
    """
    Function to train models from database.
    params: *model_name: name of the model to be trained
    """
    http_status = 200

    if model_name != "kda" and model_name != "no_kda":
        http_status = 400
    else:
        db_con = connector.DatabaseConnector()
        db_con.connect()
    
        matches = list(db_con.get())
        db_con.disconnect()
    
        # User Story 1, model no_kda
        if model_name == "no_kda":
            dataframe = build_dataframe_no_kda(matches)
        
            # Prepare features
            y_data_labels = dataframe['win']
            x_data_features = dataframe['ttlPings']
            # Reshape to 1-D array(column-vector)
            x_data_features = np.array(x_data_features).reshape(-1,1)
        
        # User Story 2/3, model kda
        else:
            dataframe_kda = build_dataframe_kda(matches)
            
            # Prepare features
            y_data_labels = dataframe_kda['win']
            x_data_features = dataframe_kda.drop(['win'], axis=1)
        
        # Train model
        x_train, x_test, y_train, y_test = train_test_split(x_data_features, y_data_labels, test_size=0.2)
        mlp_classifier = MLPClassifier(max_iter=1000).fit(x_train, y_train)
    
        # save model to database, depending on name
        if model_name == "no_kda":
            connector.save_model(mlp_classifier, "no_kda")
        else:
            connector.save_model(mlp_classifier, "kda")
    
    return http_status

In [ ]:
def build_dictionary_prediction_results(prediction: np.ndarray, prediction_proba: np.ndarray, score: float) -> dict:
    """
    Function to create the return dictionary in JSON format for prediction results.
    params: *prediction: numpy array of predictions
            *prediction_proba: numpy of probabilties of the predictions
            *score: float of prediction score
    return: JSON object of prediction results
    """
    prediction = prediction.tolist()
    prediction_proba = prediction_proba.tolist()
    result_list = []
    for i in range(len(prediction)):
        result_list.append({"predict": prediction[i], "predict_proba": prediction_proba[i]})
    return_dict = {"score": score, "results": result_list}
    return json.dumps(return_dict) #convert to correct JSON format

In [ ]:
def predict(model_name, matches):
    """
    Function to predict dataset with model.
    params: *model_name: name of the model to predict with
    params: *matches: match data to predict
    """
    http_status = 200
    prediction_results = ""

    if model_name != "kda" and model_name != "no_kda":
        http_status = 400
    else:
        # User Story 1, model no_kda for prediction
        if model_name == "no_kda":
            # Load model
            mlp_classifier = connector.load_model("no_kda")
            
            dataframe = build_dataframe_no_kda(list(matches))
            
            # Drop 'win' for prediction and save 'win' for comparison
            dataframe_predict = dataframe['ttlPings']
            win_labels = dataframe['win']
            # Reshape to 1-D array(column-vector)
            x_data_features = np.array(dataframe_predict).reshape(-1,1)
            
            # Predict
            prediction = mlp_classifier.predict(x_data_features)
            prediction_proba = mlp_classifier.predict_proba(x_data_features)
            score = mlp_classifier.score(x_data_features, win_labels)

            # Save prediction results in dictionary
            prediction_results = build_dictionary_prediction_results(prediction, prediction_proba, score)
        # User Story 2/3, model kda for prediction
        else:
            # Load model
            mlp_classifier = connector.load_model("kda")
            
            dataframe_kda = build_dataframe_kda(list(matches))
            
            # Drop 'win' for prediction and save 'win' for comparison
            win_labels = dataframe_kda['win']
            x_data_features = dataframe_kda.drop(['win'], axis=1)
    
            # Predict
            prediction = mlp_classifier.predict(x_data_features)
            prediction_proba = mlp_classifier.predict_proba(x_data_features)
            score = mlp_classifier.score(x_data_features, win_labels)

            # Save prediction results in dictionary
            prediction_results = build_dictionary_prediction_results(prediction, prediction_proba, score)
    
    return http_status, prediction_results